### Partioned Heritability for Whole Blood Gene Expression Levels 

#### Test Stratified-LDSC on a few genes using cis-eQTLs
* The window size for cis-eQTLs is defined as SNPs that are $\pm$1Mb away from TSS. 
* The window size for trans-eQTLs is defined as all SNPs located in different chromosomes.

In [2]:
import gzip
import pandas as pd

In [3]:
#build a dictionary for all the genes tested in eQTL analysis
data_dir="/project2/xuanyao/jing/data/eQTLGen/"
d={}
with open(data_dir+"gene_list.txt", 'r') as f:
    f.next()
    for line in f:
        gene = line.rstrip()
        if gene in d:
            continue
        else:
            d[gene]=[]
        

In [4]:
#obtain the coordinate of TSS for each gene
ref={}
with open(data_dir+"gencode.v19.annotation.gtf", 'r') as f:
    for line in f:
        if line.startswith("##"):
             continue 

        words = line.rstrip().split("\t")
        if words[2] == "gene":
            name = words[8].split(";")[4].split(" ")[-1].strip('\"')
            if name in ref:
                continue
            else:
                ref[name] = [words[0], words[3], words[4]]

In [25]:
for gene in d:
    if gene in ref:
        start=int(ref[gene][1])-1000000
        end=int(ref[gene][1])+1000000
        if start < 0:
            start=0
        output = ref[gene]
        output.append(start)
        output.append(end)
        d[gene]= output
        
#output coordinates for local regions
out = open(data_dir+"coordinates_for_local_regions.txt", 'w')
out.write("%s\n" % "\t".join(['gene_name', 'chr', 'pos','start','end']))
for gene in d:
    if len(d[gene])>0:
        out.write("%s\n" % "\t".join([gene]+[str(i) for i in d[gene]]))
out.close()

#### Note:
- There are 514 genes not found in the gencode reference list.

In [36]:
gene_missing=[]
for i in d:
    if len(d[i])==0:
        gene_missing.append(i)
print(len(set(gene_missing)))

514


#### Test S-LDSC on a few genes
1. Extract lines that contain all SNPs that are being tested with the gene of interest into a new file.
2. Filter out SNPs that are outside of 1Mb from TSS and the remaining ones are local SNPs. 
3. Compute LD scores for local SNPs??
4. run LDSC

In [48]:
%%bash
gunzip -c 2019-12-11-cis-eQTLsFDR-ProbeLevel-CohortInfoRemoved-BonferroniAdded.txt.gz|grep CLEC12A|wc -l

UsageError: %%bash is a cell magic, but the cell body is empty.


In [53]:
f_pheno=open(data_dir+ "./CLEC12A_local_SNPs.txt", 'w')
with open(data_dir+"CLEC12A_ciseQTLs.txt") as f:
    head=f.next().rstrip().split("\t")
    col = head.index("GeneSymbol")
    print col
    f_pheno.write("%s\n" % "\t".join(head))
    for line in f:
        words = line.rstrip().split("\t")
        snp_pos=head.index("SNPPos")
        start,end=d["CLEC12A"][2:4]
        if int(words[snp_pos]) >= start and int(words[snp_pos]) <= end:
            f_pheno.write(line)

8


In [ ]:
python /home/jinggu/github/ldsc/make_annot.py \
        --gene-set-file 

In [14]:
%%bash
#run LDSC
python ~/github/ldsc/munge_sumstats.py \
        --sumstats /project2/xuanyao/jing/data/eQTLGen/CLEC12A_local_SNPs.txt \
        --out /project2/xuanyao/jing/test_ldsc/CLEC12A_ldsc \
        --a1 AssessedAllele \
        --a2 OtherAllele \
        --p Pvalue \
        --a1-inc \
        --N-col NrSamples \
        --merge-alleles /project2/xuanyao/jing/test_ldsc/w_hm3.snplist \



*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./munge_sumstats.py \
--out /project2/xuanyao/jing/test_ldsc//CLEC12A_ldsc \
--merge-alleles /project2/xuanyao/jing/test_ldsc/w_hm3.snplist \
--a1-inc  \
--N-col NrSamples \
--a1 AssessedAllele \
--a2 OtherAllele \
--sumstats /project2/xuanyao/jing/data/eQTLGen/CLEC12A_local_SNPs.txt \
--p Pvalue 

Interpreting column names as follows:
NrSamples:	Sample size
SNP:	Variant ID (e.g., rs number)
AssessedAllele:	Allele 1, interpreted as ref allele for signed sumstat.
Pvalue:	p-Value
OtherAllele:	Allele 2, interpreted as non-ref allele for signed sumstat.

Reading list of SNPs for allele merge from /project2/xuanyao/jing/test_ldsc/w_hm3.snplist
R

In [3]:
%%bash
which python

/home/jinggu/env/ldsc/bin/python


In [6]:
%%bash
cd /project2/xuanyao/jing/test_ldsc/
python ~/github/ldsc/ldsc.py \
    --h2 CLEC12A_ldsc.sumstats.gz \
    --ref-ld-chr baseline. \
    --w-ld-chr weights_hm3_no_hla/weights. \
    --overlap-annot \
    --frqfile-chr 1000G_frq/1000G.mac5eur. \
    --print-coefficients \
    --out CLEC12A_nocoeff
    

*********************************************************************
* LD Score Regression (LDSC)
* Version 1.0.1
* (C) 2014-2019 Brendan Bulik-Sullivan and Hilary Finucane
* Broad Institute of MIT and Harvard / MIT Department of Mathematics
* GNU General Public License v3
*********************************************************************
Call: 
./ldsc.py \
--h2 CLEC12A_ldsc.sumstats.gz \
--ref-ld-chr baseline. \
--out CLEC12A_nocoeff \
--overlap-annot  \
--frqfile-chr 1000G_frq/1000G.mac5eur. \
--w-ld-chr weights_hm3_no_hla/weights. \
--print-coefficients  

Beginning analysis at Sun Jun 28 22:07:31 2020
Reading summary statistics from CLEC12A_ldsc.sumstats.gz ...
Read summary statistics for 918 SNPs.
Reading reference panel LD Score from baseline.[1-22] ...
Traceback (most recent call last):
  File "/home/jinggu/github/ldsc/ldsc.py", line 644, in <module>
    sumstats.estimate_h2(args, log)
  File "/home/jinggu/github/ldsc/ldscore/sumstats.py", line 326, in estimate_h2
    args, 

Traceback (most recent call last):
  File "/home/jinggu/github/ldsc/ldsc.py", line 644, in <module>
    sumstats.estimate_h2(args, log)
  File "/home/jinggu/github/ldsc/ldscore/sumstats.py", line 326, in estimate_h2
    args, log, args.h2)
  File "/home/jinggu/github/ldsc/ldscore/sumstats.py", line 243, in _read_ld_sumstats
    ref_ld = _read_ref_ld(args, log)
  File "/home/jinggu/github/ldsc/ldscore/sumstats.py", line 82, in _read_ref_ld
    'reference panel LD Score', ps.ldscore_fromlist)
  File "/home/jinggu/github/ldsc/ldscore/sumstats.py", line 152, in _read_chr_split_files
    out = parsefunc(_splitp(chr_arg), _N_CHR, **kwargs)
  File "/home/jinggu/github/ldsc/ldscore/parse.py", line 93, in ldscore_fromlist
    y = ldscore(fh, num)
  File "/home/jinggu/github/ldsc/ldscore/parse.py", line 137, in ldscore
    s, compression = which_compression(first_fh)
  File "/home/jinggu/github/ldsc/ldscore/parse.py", line 43, in which_compression
    raise IOError('Could not open {F}[./gz/bz2]'

In [16]:
tbl=pd.read_table("/project2/xuanyao/jing/test_ldsc/CLEC12A_baseline.results")
tbl.head()

,Category,Prop._SNPs,Prop._h2,Prop._h2_std_error,Enrichment,Enrichment_std_error,Enrichment_p,Coefficient,Coefficient_std_error,Coefficient_z-score
0,base_0,1.000000,1.000000,0.000063,1.000000,0.000063,NaN,-0.000043,0.000030,-1.445854
1,Coding_UCSC_0,0.014658,-0.430966,18.456212,-29.401084,1259.108691,0.550893,-0.000098,0.000091,-1.079422
2,Coding_UCSC.extend.500_0,0.064555,0.925546,29.571643,14.337226,458.081516,0.300544,0.000049,0.000022,2.208477
3,Conserved_LindbladToh_0,0.026063,0.655425,9.387973,25.148205,360.209902,0.821024,0.000047,0.000156,0.298041
4,Conserved_LindbladToh.extend.500_0,0.332514,-0.439661,52.695949,-1.322233,158.477369,0.749612,-0.000010,0.000015,-0.651321


,Category,Prop._SNPs,Prop._h2,Prop._h2_std_error,Enrichment,Enrichment_std_error,Enrichment_p,Coefficient,Coefficient_std_error,Coefficient_z-score
0,base_0,1.000000,1.000000,0.000063,1.000000,0.000063,NaN,-0.000043,0.000030,-1.445854
1,Coding_UCSC_0,0.014658,-0.430966,18.456212,-29.401084,1259.108691,0.550893,-0.000098,0.000091,-1.079422
2,Coding_UCSC.extend.500_0,0.064555,0.925546,29.571643,14.337226,458.081516,0.300544,0.000049,0.000022,2.208477
3,Conserved_LindbladToh_0,0.026063,0.655425,9.387973,25.148205,360.209902,0.821024,0.000047,0.000156,0.298041
4,Conserved_LindbladToh.extend.500_0,0.332514,-0.439661,52.695949,-1.322233,158.477369,0.749612,-0.000010,0.000015,-0.651321
